### And welcome to Week 4, Day 3 - more LangGraph..

In [4]:
from typing import Annotated
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain_openai import ChatOpenAI
from typing import TypedDict


In [5]:
# Our favorite first step! Crew was doing this for us, by the way.
load_dotenv(override=True)


True

### First, let's go set up LangSmith!

https://langsmith.com

### Next, here is a useful function in LangChain community:

In [6]:
from langchain_community.utilities import GoogleSerperAPIWrapper
# This is a utility function from langchain community
# IMPORTANT: a wrapper function encapsulate a complex logic to provide a simple use of different services.
# It's very useful in the agents architecture

serper = GoogleSerperAPIWrapper()
serper.run("What is the capital of France?")

'Paris is the capital and largest city of France, with an estimated city population of 2,048,472 in an area of 105.4 km2 (40.7 sq mi), and a metropolitan ... Paris is the capital and most populous city of France. Situated on the Seine River, in the north of the country, it is in the centre of the Île-de-France ... Paris is the capital of France, the largest country of Europe with 550 000 km2 (65 millions inhabitants). Paris has 2.234 million inhabitants end 2011. Paris, city and capital of France, located along the Seine River, in the north-central part of the country. Paris is one of the world\'s most ... Paris is the city of romance par excellence, the fashion capital and the best example of French art de vivre. Exploring Paris is an essential rite of passage ... The capital of France has been Paris since its liberation in 1944. Paris, the capital and largest city of France, is located in the northern part of the country on the banks of the Seine River. Paris, the capital of France, 

### Now here is a LangChain wrapper class for converting functions into Tools

In [7]:
from langchain.agents import Tool

tool_search =Tool(
        name="search",
        func=serper.run,
        description="Useful for when you need more information from an online search"
    )

# IMPORTANT: with this achitecture we can wrap the serper.run wrapper into a Tool wrapper to easily transform the Serper wrapper into a tool
# This approach allow to create automatically the JSON schema

### Now we can try out the tool the langchain way

In [8]:
tool_search.invoke("What is the capital of France?")
# With this approach we can invoke the tool

"Paris is the capital and largest city of France, with an estimated city population of 2,048,472 in an area of 105.4 km2 (40.7 sq mi), and a metropolitan ... Paris is the capital and most populous city of France. Situated on the Seine River, in the north of the country, it is in the centre of the Île-de-France ... Paris is the capital of France, the largest country of Europe with 550 000 km2 (65 millions inhabitants). Paris has 2.234 million inhabitants end 2011. Paris, city and capital of France, located along the Seine River, in the north-central part of the country. Paris is one of the world's most ... The capital of France has been Paris since its liberation in 1944. It is home to iconic landmarks like the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum, which houses the famous Mona Lisa. Known for ... Paris , the capital and largest city of France, is located in the northern part of the country on the banks of the Seine River . Paris is the city of romance par excellence

### And now let's write a tool ourselves

We'll pick a familiar one

In [9]:
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user"""
    requests.post(pushover_url, data = {"token": pushover_token, "user": pushover_user, "message": text})

In [10]:
# NB: once created the push funciton we transform it in a tool by using the Tool wrapper as done before with serper
tool_push = Tool(
        name="send_push_notification",
        func=push,
        description="useful for when you want to send a push notification"
    )

tool_push.invoke("Hello, me")

### Back to the Graph from yesterday

One small change - using TypedDict instead of BaseModel for the State object

When we implement tools, we always need to make 2 changes to the code:

1. Changes to provide the tools to OpenAI in json when we make the call

2. Changes to handle the results back: look for the model staying that the finish_reason=="tool_calls" and then retrieve the call, run the function, provide the results.

NOTE: in the first week we used globals() to access the tool functions by name in the global environment.

### Bring them together

In [11]:
# Here we create the list of tools to be passed to the agent
tools = [tool_search, tool_push]

In [12]:
# Step 1: Define the State object
# To be noted: we're using TypeDict instead of BaseModel.
# The syntax is the same, we still use Annotated and the reducer function "add_messages"
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [13]:
# Step 2: Start the Graph Builder with this State class
graph_builder = StateGraph(State)

In [14]:
# This is different:

llm = ChatOpenAI(model="gpt-4o-mini") # This is the langChain wrapper to call OpenAI LLM model
# bind_tools è un metodo del wrapper ChatOpenAI che:
# 1. Prende una lista di tools/funzioni come parametro
# 2. Modifica il prompt per includere le descrizioni dei tools nel formato JSON richiesto da OpenAI
# 3. Gestisce automaticamente il parsing delle risposte quando il modello decide di usare un tool
# 4. Ritorna una nuova istanza del wrapper configurata con i tools forniti
llm_with_tools = llm.bind_tools(tools)

In [15]:
# Step 3: Create a Node
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]} # In our node we invoke the LLM with tools (not the simple LLM)

graph_builder.add_node("chatbot", chatbot) # This is the first node created "chatbot"
graph_builder.add_node("tools", ToolNode(tools=tools)) # ATTENTION: this is special kind of node called "tools" representing the tools node
# The tools node see if in the messages passed by the user there's the need to use one of the tools, if so it runs them

In [16]:
# Step 4: Create Edges

# NOTA IMPORTANTE: Per collegare i tools dobbiamo usare edges condizionali invece di edges normali
# Questo perché non possiamo sapere a priori se un tool verrà effettivamente utilizzato durante l'esecuzione
# tools_condition è una funzione che determina se il messaggio del chatbot richiede l'uso di un tool
# In pratica tools_condition corrisponde al blocco "if statement" che determina se usare un tool o no
graph_builder.add_conditional_edges("chatbot", tools_condition, "tools") # Linea tratteggiata da chatbot a tools (condizionale)

# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot") # linea solida da tools a chatbot perche una volta che si arriva a tools il processo deve sempre tornare indietro
graph_builder.add_edge(START, "chatbot") # linea solida da start a chatbot

# NOTA: LangGraph crea automaticamente un edge condizionale da chatbot a END
# Questo perché quando il chatbot non ha bisogno di usare tools (tools_condition è falsa)
# significa che ha completato la sua risposta e può terminare il flusso
# Non è quindi necessario creare esplicitamente questo edge

In [ ]:
# Step 5: Compile the Graph
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

### That's it! And, let's do this:

In [3]:
def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]})
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

NameError: name 'gr' is not defined

## OK it's time to add Memory!

### BUT WAIT!

We have this whole Graph maintaining the state and appending to the state.

Why isn't this handling memory?

### This is a crucial point for understanding LangGraph

> A super-step can be considered a single iteration over the graph nodes. Nodes that run in parallel are part of the same super-step, while nodes that run sequentially belong to separate super-steps.


One "Super-Step" of the graph represents one invocation of passing messages between agents.

In idomatic LangGraph, you call invoke to run your graph for each super-step; for each interaction.

The reducer handles state updates automatically within one super-step, but not between them.

That is what checkpointing achieves.

In [18]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
# Steps 1 and 2
graph_builder = StateGraph(State)


# Step 3
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Step 5
graph = graph_builder.compile(checkpointer=memory) # we add this method to allow state being saved in the checkpoints
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# We must create this config varialble to be passed below in the invocation of the graph to save states in a memory slot
config = {"configurable": {"thread_id": "1"}} # By changing the thread_id number we change the memory slot

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

In [ ]:
graph.get_state(config)
# This will return the state of the graph

In [ ]:
# This will return the entire history of the graph starting from the most recent state to the older
# This way LangGraph allow to step back in time by passing the specific checkpoint ID
list(graph.get_state_history(config))

### LangGraph gives you tools to set the state back to a prior point in time, to branch off:

## This is called the Time Travel
```
config = {"configurable": {"thread_id": "1", "checkpoint_id": ...}}
graph.invoke(None, config=config)
```

And this allows you to build stable systems that can be recovered and rerun from any prior checkpoint.

### And now let's store in SQL

### And this is the power of LangGraph.

In [21]:
# Here we use SqliteSver instead of MemorySaver
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

db_path = "memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(conn)

In [ ]:
# Steps 1 and 2
graph_builder = StateGraph(State)


# Step 3
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Step 5
graph = graph_builder.compile(checkpointer=sql_memory) # IMPORTANT: here we use the instantiation of SQLite
display(Image(graph.get_graph().draw_mermaid_png()))
 

In [23]:
config = {"configurable": {"thread_id": "3"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


{'messages': [HumanMessage(content='hi there', additional_kwargs={}, response_metadata={}, id='d6144090-9963-438c-b21b-3fb4f06edbc0'), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 90, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BInN7JUnEYNTQqokCF3eihp5oKP0V', 'finish_reason': 'stop', 'logprobs': None}, id='run-4b488710-73ac-435e-a944-913a0c812d08-0', usage_metadata={'input_tokens': 90, 'output_tokens': 11, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content="what's my name", additional_kwargs

### Persistent memory
Note memory is stored persistently even restarting the Kernel since the information aree sterein in files in the project tree.
Look at the project tree to see:
* memory.db
* memory.db-shm
* memory.db-wal